#EPH - Individuos


In [ ]:
import sys
import os
import csv
from pathlib import Path

Antes de comenzar, se establece la ruta a la fuente para no tener problemas en la importación de módulos.


In [ ]:
# Ajustar la ruta para que Python reconozca la carpeta "src"
ruta_proyecto = os.path.abspath(os.path.join(".."))  
sys.path.append(ruta_proyecto) 
# Verificación (debe mostrar la ruta a "src/")
print("Ruta de 'src':", os.path.join(ruta_proyecto, "src"))

## Unificación del archivo

In [ ]:
from src.utils.helpers import procesar_archivos,guardar_en_txt
# Ruta los archivos 
ruta = "../data/raw"
ruta_destino = "../data/clean"

# Procesar y guardar individuos
individual = procesar_archivos(ruta, tipo="individual")
guardar_en_txt(individual, ruta_destino, "individuos_unificado.txt")